In [27]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import model_from_json

import pandas as pd
import numpy as np
import pandas_ta as ta

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

import os
import sys

In [28]:
def calculate_rsi(df: pd.DataFrame, length: int = 14):
    if not isinstance(length, int):
        raise Exception("length must be an integer!")
    df[f'RSI_{length}'] = ta.rsi(df['Close'], length=length)

def calculate_sma(df: pd.DataFrame, length: int = 50):
    """
    Wrapper function for sma
    :param df:
    :param length:
    :return:
    """
    if not isinstance(length, int):
        raise Exception("length must be an integer!")
    df[f'SMA_{length}'] = ta.sma(df['Close'], length=length)

def calculate_macd(df: pd.DataFrame, fast: int = 12, slow: int = 26, signal: int = 9):
    """
    Wrapper function for MACD
    :param df:
    :param fast:
    :param slow:
    :param signal:
    :return:
    """
    if not isinstance(fast, int):
        raise Exception("fast must be an integer!")
    if not isinstance(slow, int):
        raise Exception("slow must be an integer!")
    if not isinstance(signal, int):
        raise Exception("signal must be an integer!")
    df[f'MACD_{fast}_{slow}_{signal}'] = ta.macd(close=df['Close'], fast=fast, slow=slow, signal=signal)[
        f'MACD_{fast}_{slow}_{signal}']

In [29]:
df = pd.read_parquet('../ProcessingData/BTC Minute CSVs/BTCUSER_all_processed.parquet')
# df.head()
indexes = [i for i in range(0, len(df)) if i % (30) == 0]
temp = df.iloc[indexes]
temp = temp.loc[df['Date'] > '2019-06-01 00:00:00']
df = temp

In [30]:
#df = pd.read_csv("..//ProcessingData//BTC Minute CSVs//BTCUSER_all_processed.csv")
# df = pd.read_csv("gemini_BTCUSD_2021_1min.csv")

# df = macd(df)
# df = moving_average(df, 14)
# df = df.join(relative_strength_index(df))
calculate_rsi(df)
calculate_sma(df)
calculate_macd(df)
df = df[50:]

In [31]:
df

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,RSI_14,SMA_50,MACD_12_26_9
1873230,1559437800000,2019-06-02 01:10:00,BTCUSD,8601.24,8601.24,8599.77,8599.77,0.045667,58.397772,8545.1578,12.764778
1873260,1559439600000,2019-06-02 01:40:00,BTCUSD,8571.90,8571.90,8571.90,8571.90,0.199783,52.510880,8546.2718,11.942756
1873290,1559441400000,2019-06-02 02:10:00,BTCUSD,8585.14,8585.14,8585.14,8585.14,0.000068,54.839936,8547.6506,12.218804
1873320,1559443200000,2019-06-02 02:40:00,BTCUSD,8580.30,8580.71,8580.30,8580.71,0.009145,53.887633,8548.9408,11.942445
1873350,1559445000000,2019-06-02 03:10:00,BTCUSD,8564.08,8564.08,8556.96,8556.96,0.228999,48.977228,8549.7560,9.695242
...,...,...,...,...,...,...,...,...,...,...,...
3134280,1638653820000,2021-12-04 21:37:00,BTCUSD,48916.50,48945.69,48908.45,48945.69,0.081272,51.642789,49635.0744,11.085700
3134310,1638655620000,2021-12-04 22:07:00,BTCUSD,48705.18,48740.92,48705.18,48738.07,0.580081,49.166676,49532.8408,5.963928
3134340,1638657420000,2021-12-04 22:37:00,BTCUSD,49133.94,49133.94,49059.62,49059.62,0.412475,52.930781,49434.8950,27.533890
3134370,1638659220000,2021-12-04 23:07:00,BTCUSD,49411.09,49517.02,49390.61,49484.96,4.048899,57.422063,49351.1378,78.049914


In [32]:

def create_dataset(df, lb, steps_into_future):
    x = []
    y = []
    for i in range(lb, df.shape[0] - steps_into_future):
        x.append(df[i - lb:i, 0])
        y.append(df[i:i + steps_into_future, 0])
    x = np.array(x)
    y = np.array(y)
    return x, y


In [33]:
def get_sets(df, column_name, scale = True):
    values = df[column_name].values
    values = values.reshape(-1, 1)
    dataset_train = np.array(values[:int(values.shape[0] * 0.8)])
    dataset_test = np.array(values[int(values.shape[0] * 0.8) - 50:])
    scaler = None
    if scale:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(dataset_train)
        dataset_train = scaler.transform(dataset_train)
        dataset_test = scaler.transform(dataset_test)
    x_train, y_train = create_dataset(dataset_train, 50, 50)
    x_test, y_test = create_dataset(dataset_test, 50, 50)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    return x_train, y_train, x_test, y_test, scaler

In [34]:
x_train_prices, y_train, x_test_prices, y_test, scaler_prices = get_sets(df, 'Open', True)

In [35]:
x_train_rsi, _, x_test_rsi, _ , _ = get_sets(df, 'RSI_14' , False)

In [36]:
x_train_sma, _, x_test_sma, _, _  = get_sets(df, 'SMA_50', True)

In [37]:
x_train_macd, _, x_test_macd, _, _  = get_sets(df, 'MACD_12_26_9', True)

In [38]:
from tensorflow.keras.optimizers import Adam
from keras import Input
from keras.models import Model
from keras.layers import Concatenate

#using functional API

input_prices = Input(shape=(50, 1), dtype='float32')
prices_layer_1 = LSTM(units=96, return_sequences=True)(input_prices)
prices_layer_2 = Dropout(0.2)(prices_layer_1)
prices_layer_3 = LSTM(units=96, return_sequences=True)(prices_layer_2)
prices_layer_4 = Dropout(0.2)(prices_layer_3)



input_rsi = Input(shape=(50, 1), dtype='float32')
rsi_layer_1 = LSTM(units=96, return_sequences=True)(input_rsi)
rsi_layer_2 = Dropout(0.2)(rsi_layer_1)
rsi_layer_3 = LSTM(units=96, return_sequences=True)(rsi_layer_2)
rsi_layer_4 = Dropout(0.2)(rsi_layer_3)


input_sma = Input(shape=(50, 1), dtype='float32')
sma_layer_1 = LSTM(units=96, return_sequences=True)(input_sma)
sma_layer_2 = Dropout(0.2)(sma_layer_1)
sma_layer_3 = LSTM(units=96, return_sequences=True)(sma_layer_2)
sma_layer_4 = Dropout(0.2)(sma_layer_3)



input_macd = Input(shape=(50, 1), dtype='float32')
macd_layer_1 = LSTM(units=96, return_sequences=True)(input_macd)
macd_layer_2 = Dropout(0.2)(macd_layer_1)
macd_layer_3 = LSTM(units=96, return_sequences=True)(macd_layer_2)
macd_layer_4 = Dropout(0.2)(macd_layer_3)



input = Concatenate()([prices_layer_4, rsi_layer_4, sma_layer_4, macd_layer_4])


x = LSTM(units=96, return_sequences=True)(input)
x = Dropout(0.2)(x)
x = LSTM(units=96)(x)
x = Dropout(0.2)(x)
x = Dense(50)(x)

model = Model(inputs=[input_prices, input_rsi, input_sma, input_macd], outputs=x)
model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')

In [ ]:
train_list = [x_train_prices, x_train_rsi, x_train_sma, x_train_macd]
test_list = [x_test_prices, x_test_rsi, x_test_sma, x_test_macd]
for i in range(0, 100):
    model.fit(train_list, y_train, epochs=20, validation_data=(test_list, y_test), batch_size=32)
    model.save(f'stock_prediction_merged_v3_{i}_steps')


Epoch 1/20
1048/1048 [==============================] - 465s 411ms/step - loss: 0.0041 - val_loss: 8.0379e-04
Epoch 2/20
1048/1048 [==============================] - 346s 330ms/step - loss: 0.0012 - val_loss: 5.4926e-04
Epoch 3/20
1048/1048 [==============================] - 347s 331ms/step - loss: 8.4809e-04 - val_loss: 5.3851e-04
Epoch 4/20
1048/1048 [==============================] - 388s 370ms/step - loss: 7.3281e-04 - val_loss: 5.4427e-04
Epoch 5/20
1048/1048 [==============================] - ETA: 0s - loss: 6.6631e-04

In [ ]:
predictions = model.predict(test_list)

# predictions = scaler.inverse_transform(predictions)
#
# fig, ax = plt.subplots(figsize=(8, 4))
# plt.plot(df, color='red',  label="True Price")
# ax.plot(range(len(y_train)+50, len(y_train)+50+len(predictions)), predictions, color='blue', label='Predicted Testing Price')
# plt.legend()
# plt.show()
#
#
#
# y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))
#
# fig, ax = plt.subplots(figsize=(8, 4))
# ax.plot(y_test_scaled, color='red', label='True Testing Price')
# plt.plot(predictions, color='blue', label='Predicted Testing Price')
# plt.legend()
# plt.show()

In [ ]:
y_test

In [ ]:
predictions

In [ ]:
# y_test_scaled = scaler.inverse_transform(y_test[0].reshape(-1, 1))
# predictions_scaled = scaler.inverse_transform(predictions[0].reshape(-1, 1))
all_test_lines = np.concatenate(y_test)
all_test_predictions = np.concatenate(predictions)

In [ ]:
y_test_scaled = scaler_prices.inverse_transform(all_test_lines.reshape(-1, 1))
predictions_scaled = scaler_prices.inverse_transform(all_test_predictions.reshape(-1, 1))

In [ ]:
plt.plot(y_test_scaled, color='red', label='Actual Prices')
plt.plot(predictions_scaled, color='blue', label='Predicted Prices')